In [ ]:
#To identify which sub-networks i.e. communities, are of interest, we'll calculate the correlation between the networks and the
#relevant clinical metadata variables

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from sklearn.decomposition import PCA
from scipy.stats import pearsonr
from statsmodels.stats.multitest import multipletests
import pickle
import seaborn as sns

In [ ]:
#Read in required data

In [ ]:
#Load in the required data
datExpr = pd.read_csv('/data/recode/mic_datExpr_pseudobulk.csv', index_col = 0)
metadata = pd.read_csv('/da/recode/mic_metadata_pseudobulk.csv', index_col = 0)

In [ ]:
datExpr

In [ ]:
metadata

In [ ]:
gene_names = datExpr.columns
gene_names

In [ ]:
with open('/shared/as8020/recode/separated_communities.pkl', 'rb') as file:
        separated_communities = pickle.load(file)

In [ ]:
print(separated_communities)

In [ ]:
#Module eigengene calculation
#Module eigengene calculation is a concept used in the analysis of gene expression data.

#Module/sub-network: A module refers to a group of genes that exhibit similar expression patterns across samples. Modules are often identified using clustering algorithms applied to gene expression data.
#Eigengene: An eigengene represents the overall expression profile of a module. It is calculated as the first principal component of the gene expression profiles within the module. Essentially, the eigengene captures the main axis of variation or the common expression pattern shared by the genes within the module.

#We will be using module eigengenes as representations for further downstream analysis.

In [ ]:
# Initialise a DataFrame to store module eigengenes
module_eigengenes = pd.DataFrame(index=datExpr.index)

# Calculate the module eigengene for each community
for i, community in enumerate(separated_communities):
    community_genes = [gene for gene in community if gene in gene_names]
    if community_genes:
        community_expr = datExpr[community_genes]
        pca = PCA(n_components=1)
        eigengene = pca.fit_transform(community_expr)
        module_eigengenes[f'Module_{i+1}'] = eigengene[:, 0]

In [ ]:
metadata

In [ ]:
metadata2 = metadata.drop(columns=['donor_id.1','nUMI', 'nGene','seurat.subclusters'])

In [ ]:
# Merge module eigengenes with metadata
merged_data = pd.concat([metadata2, module_eigengenes], axis=1)

In [ ]:
merged_data

In [ ]:
# Calculate Pearson correlation between module eigengenes and metadata
correlation_matrix2 = merged_data.corr(method='pearson')

In [ ]:
correlation_matrix2

In [ ]:
#The correlation matrix needs to be reformatted into the correct format:
correlation_matrix3 = correlation_matrix2.drop(['BraakStage', 'development_stage', 'AD', 'Normal'])

In [ ]:
correlation_matrix3

In [ ]:
correlation_matrix3 = correlation_matrix3.drop(correlation_matrix3.columns[4:], axis=1)

In [ ]:
correlation_matrix3

In [ ]:
#We now need to calculate the p-values for the correlations on the merged data.
# Initialize an empty DataFrame to store p-values
module_p_values = pd.DataFrame(index=module_eigengenes.columns, columns=metadata2.columns)

# Calculate p-values for correlations between module eigengenes and metadata
for module in module_eigengenes.columns:
    for metadata_column in metadata2.columns:
        # Calculate correlation coefficient and p-value
        correlation_coefficient, p_value = pearsonr(module_eigengenes[module], metadata2[metadata_column])
        # Store p-value in the DataFrame
        module_p_values.loc[module, metadata_column] = p_value

print(module_p_values)

In [ ]:
#We now need to calculate the adjusted p-values.
# Calculate adjusted p-values using FDR correction
adjusted_p_values = pd.DataFrame(index=module_p_values.index, columns=module_p_values.columns)
for column in module_p_values.columns:
    p_values = module_p_values[column].astype(float)
    # Perform FDR correction
    _, adj_p_values, _, _ = multipletests(p_values, method='fdr_bh')
    adjusted_p_values[column] = adj_p_values

print(adjusted_p_values)

In [ ]:
#We want a method to highlight interesting modules.
#For this purpose we will be creating a heatmap of the correlations and associated adjusted p-values between the modules and the metadata.
# Function to annotate heatmap with correlation values and p-values
def annotate_heatmap_with_p_values(ax, correlation_matrix, p_values, threshold=0.05):
    stars = np.empty(p_values.shape, dtype='<U2')
    stars[p_values > threshold] = ''
    stars[p_values <= threshold] = '*'
    for i in range(correlation_matrix.shape[0]):
        for j in range(correlation_matrix.shape[1]):
            # Format annotation string with correlation value
            annotation_corr = f"{correlation_matrix.iloc[i, j]:.2f}"
            # Format p-value in brackets
            annotation_p_value = f"({p_values.iloc[i, j]:.2f})"
            # Add star for significant p-values
            if p_values.iloc[i, j] <= threshold:
                annotation_p_value += '*'
            # Add annotations
            ax.text(j+0.5, i+0.4, annotation_corr, ha='center', va='center', color='black')
            ax.text(j+0.5, i+0.6, annotation_p_value, ha='center', va='center', color='black')


In [ ]:
# Visualise the correlation matrix as a heatmap with correlation values and p-values
plt.figure(figsize=(10, 8))
heatmap = sns.heatmap(correlation_matrix3, annot=False, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation between Module Eigengenes and Metadata')

# Annotate heatmap with correlation values and p-values
annotate_heatmap_with_p_values(heatmap, correlation_matrix3, adjusted_p_values)

plt.show()

In [ ]:
#Exercise Questions
#1. What is a correlation and why is it used in gene co-expression analysis?
#2. Why are adjusted p-values used instead of p-values?
#3. What is fdr_bh and explain it fully?
#4. Based on the heatmap, what can you summarise?

In [ ]:
#Answers
#1.In the context of gene co-expression analysis, correlation refers to the statistical measure of the strength and direction of association between the expression levels of two genes across a set of samples or conditions. It quantifies the degree to which the expression patterns of two genes are similar or dissimilar across different biological contexts.
#2.Adjusted p-values are used instead of raw p-values in statistical hypothesis testing to correct for multiple comparisons. When conducting multiple statistical tests simultaneously (such as testing multiple genes in gene expression analysis or multiple variables in genomics studies), the probability of obtaining false positive results increases. This increased probability arises due to the cumulative effect of conducting multiple tests, leading to an inflated Type I error rate (false positives).
#3.FDR_BH stands for False Discovery Rate (FDR) control using the Benjamini-Hochberg (BH) procedure. It is a statistical method used to control the proportion of false positives among all significant results when conducting multiple hypothesis tests simultaneously.
#4.Modules 1, 2, 3, 5 and 6 appear to have the strongest, significant correlations with Braak stage, as well as relatively strong correlations with development stage and AD clinical metadata variables, highlighting them as being of interest in context of disease.
#WhereasModules 7, 8 and 9 seem to be slightly associated with Normal category.